In [5]:
import h5py
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader
import numpy as np
import cv2 as cv

import model
import synth_text_dataset
from transforms import img_transform, char_transform, labels_transform

%matplotlib inline

In [2]:
filename = 'Project/SynthText_train.h5'
num_of_images = 998
train_dataset = synth_text_dataset.SynthTextCharactersDatasetRAM(filename, full_image_transform=img_transform, on_get_item_transform=char_transform, target_transform=labels_transform, end_idx=int(0.8*num_of_images))
test_dataset = synth_text_dataset.SynthTextCharactersDatasetRAM(filename, full_image_transform=img_transform, on_get_item_transform=char_transform, target_transform=labels_transform, start_idx=int(0.8*num_of_images))

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, shuffle=True)

In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
init_shape = (200, 100)
classifier = model.FontClassifierModel(init_shape, 1)
lr = 1e-3
epochs = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classifier.parameters(), lr=lr)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, classifier, loss_fn, optimizer)
    test_loop(test_dataloader, classifier, loss_fn)
print("Done!")